In [1]:
import pandas as pd
import numpy as np

from keras import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

import optuna

c:\Users\yveem\miniconda3\envs\machine_learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### **Getting training & validation data**

In [2]:
df_train = pd.read_csv('../databases/training.csv')
df_valid = pd.read_csv('../databases/validation.csv')

In [3]:
X_train, y_train = df_train.drop(['IND_BOM_1_1'], axis=1), df_train['IND_BOM_1_1']
X_val, y_val = df_valid.drop(['IND_BOM_1_1'], axis=1), df_valid['IND_BOM_1_1']

In [4]:
X_train = np.array(X_train)
X_val = np.array(X_val)

#### **Parameters selection**

In [5]:
params = {
    'hidden_layer_sizes': [32, 128, 256],
    'hidden_layers': {
        'low': 1,
        'high': 2
    },
    'alpha': [0.0001, 0.01],
    'max_iter': [2000, 5000, 10000],
    'batch_size': [32, 64, 128],
    'activation': ['tanh', 'relu', 'sigmoid'],
    'optimizer': ['SGD', 'Adam'],
    'output_activation': ['sigmoid', 'softmax']
}

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True,
    min_delta=0.001
)

def objective(trial):
    model = Sequential()
    input_dimension = X_train.shape[1]
    
    model.add(
        Dense(
            input_dim=input_dimension,
            units=trial.suggest_categorical(
                'hidden_layer_sizes',
                params['hidden_layer_sizes']
            ),
            activation=trial.suggest_categorical(
                'activation',
                params['activation']
            )
        )
    )

    hidden_layers = trial.suggest_int('hidden_layers', params['hidden_layers']['low'], params['hidden_layers']['high'])
    
    for _ in range(hidden_layers):
        model.add(
            Dense(
                input_dim=input_dimension,
                units=trial.suggest_categorical(
                    'hidden_layer_sizes',
                    params['hidden_layer_sizes']
                ),
                activation=trial.suggest_categorical(
                    'activation',
                    params['activation']
                )
            )
        )
    
    model.add(
        Dense(
            units=1,
            activation=trial.suggest_categorical(
                'output_activation',
                params['output_activation']
            )
        )
    )

    alpha = trial.suggest_categorical('alpha', params['alpha'])
    optimizer_name = trial.suggest_categorical('optimizer', params['optimizer'])

    if optimizer_name == 'SGD':
        optimizer = SGD(learning_rate=alpha)
    elif optimizer_name == 'Adam':
        optimizer = Adam(learning_rate=alpha)

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    model.fit(
        X_train,
        y_train,
        batch_size=trial.suggest_categorical(
            'batch_size',
            params['batch_size']
        ),
        epochs=trial.suggest_categorical(
            'max_iter',
            params['max_iter']
        ),
        validation_data=(X_val, y_val),
        callbacks=[early_stopping]
    )

    loss, accuracy = model.evaluate(X_val, y_val)

    return loss

In [6]:
n_trials = 10

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials)

[I 2023-03-29 14:19:44,543] A new study created in memory with name: no-name-d5243979-45b3-4e9f-a017-82e6bf7ee3f1


Epoch 1/10000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.6437 - accuracy: 0.6555 - val_loss: 0.6434 - val_accuracy: 0.6554
Epoch 2/10000
1977/1977 [==============================] - 5s 3ms/step - loss: 0.6427 - accuracy: 0.6555 - val_loss: 0.6423 - val_accuracy: 0.6554
Epoch 3/10000
1977/1977 [==============================] - 5s 3ms/step - loss: 0.6416 - accuracy: 0.6555 - val_loss: 0.6411 - val_accuracy: 0.6554
Epoch 4/10000
1977/1977 [==============================] - 5s 3ms/step - loss: 0.6403 - accuracy: 0.6555 - val_loss: 0.6397 - val_accuracy: 0.6554
Epoch 5/10000
1977/1977 [==============================] - 5s 2ms/step - loss: 0.6387 - accuracy: 0.6555 - val_loss: 0.6379 - val_accuracy: 0.6554
Epoch 6/10000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.6368 - accuracy: 0.6555 - val_loss: 0.6358 - val_accuracy: 0.6554
Epoch 7/10000
1977/1977 [==============================] - 7s 4ms/step - loss: 0.6343 - accuracy: 0.6555 - val_loss: 0

[I 2023-03-29 14:23:59,346] Trial 0 finished with value: 0.602855920791626 and parameters: {'hidden_layer_sizes': 32, 'activation': 'sigmoid', 'hidden_layers': 1, 'output_activation': 'sigmoid', 'alpha': 0.01, 'optimizer': 'SGD', 'batch_size': 128, 'max_iter': 10000}. Best is trial 0 with value: 0.602855920791626.


Epoch 1/2000
1977/1977 [==============================] - 10s 5ms/step - loss: 0.6592 - accuracy: 0.6555 - val_loss: 0.6477 - val_accuracy: 0.6554
Epoch 2/2000
1977/1977 [==============================] - 9s 5ms/step - loss: 0.6457 - accuracy: 0.6555 - val_loss: 0.6447 - val_accuracy: 0.6554
Epoch 3/2000
1977/1977 [==============================] - 9s 4ms/step - loss: 0.6441 - accuracy: 0.6555 - val_loss: 0.6438 - val_accuracy: 0.6554
Epoch 4/2000
1977/1977 [==============================] - 8s 4ms/step - loss: 0.6432 - accuracy: 0.6555 - val_loss: 0.6429 - val_accuracy: 0.6554
Epoch 5/2000
1977/1977 [==============================] - 9s 4ms/step - loss: 0.6424 - accuracy: 0.6555 - val_loss: 0.6421 - val_accuracy: 0.6554
Epoch 6/2000
1977/1977 [==============================] - 8s 4ms/step - loss: 0.6415 - accuracy: 0.6555 - val_loss: 0.6413 - val_accuracy: 0.6554
Epoch 7/2000
1977/1977 [==============================] - 10s 5ms/step - loss: 0.6408 - accuracy: 0.6555 - val_loss: 0.6406

[I 2023-03-29 14:43:15,481] Trial 1 finished with value: 0.6117700338363647 and parameters: {'hidden_layer_sizes': 256, 'activation': 'relu', 'hidden_layers': 2, 'output_activation': 'softmax', 'alpha': 0.0001, 'optimizer': 'SGD', 'batch_size': 128, 'max_iter': 2000}. Best is trial 0 with value: 0.602855920791626.


...

[I 2023-03-29 15:54:01,470] Trial 7 finished with value: 0.5933857560157776 and parameters: {'hidden_layer_sizes': 32, 'activation': 'relu', 'hidden_layers': 2, 'output_activation': 'softmax', 'alpha': 0.01, 'optimizer': 'SGD', 'batch_size': 128, 'max_iter': 5000}. Best is trial 4 with value: 0.5899200439453125.


Epoch 1/10000
1977/1977 [==============================] - 8s 4ms/step - loss: 0.6162 - accuracy: 0.6630 - val_loss: 0.6062 - val_accuracy: 0.6687
Epoch 2/10000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.6018 - accuracy: 0.6725 - val_loss: 0.6028 - val_accuracy: 0.6714
Epoch 3/10000
1977/1977 [==============================] - 5s 3ms/step - loss: 0.5989 - accuracy: 0.6753 - val_loss: 0.6005 - val_accuracy: 0.6730
Epoch 4/10000
1977/1977 [==============================] - 5s 3ms/step - loss: 0.5972 - accuracy: 0.6761 - val_loss: 0.5990 - val_accuracy: 0.6740
Epoch 5/10000
1977/1977 [==============================] - 5s 3ms/step - loss: 0.5959 - accuracy: 0.6774 - val_loss: 0.5982 - val_accuracy: 0.6749
Epoch 6/10000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.5947 - accuracy: 0.6786 - val_loss: 0.5972 - val_accuracy: 0.6761
Epoch 7/10000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.5935 - accuracy: 0.6791 - val_loss: 0

[I 2023-03-29 15:57:14,276] Trial 8 finished with value: 0.5913050770759583 and parameters: {'hidden_layer_sizes': 32, 'activation': 'tanh', 'hidden_layers': 2, 'output_activation': 'sigmoid', 'alpha': 0.0001, 'optimizer': 'Adam', 'batch_size': 128, 'max_iter': 10000}. Best is trial 4 with value: 0.5899200439453125.


Epoch 1/10000
7906/7906 [==============================] - 26s 3ms/step - loss: 0.6641 - accuracy: 0.6555 - val_loss: 0.6440 - val_accuracy: 0.6554
Epoch 2/10000
7906/7906 [==============================] - 24s 3ms/step - loss: 0.6440 - accuracy: 0.6555 - val_loss: 0.6440 - val_accuracy: 0.6554
Epoch 3/10000
7906/7906 [==============================] - 23s 3ms/step - loss: 0.6440 - accuracy: 0.6555 - val_loss: 0.6440 - val_accuracy: 0.6554
Epoch 4/10000
7906/7906 [==============================] - 25s 3ms/step - loss: 0.6440 - accuracy: 0.6555 - val_loss: 0.6440 - val_accuracy: 0.6554
Epoch 5/10000
7906/7906 [==============================] - 26s 3ms/step - loss: 0.6440 - accuracy: 0.6555 - val_loss: 0.6440 - val_accuracy: 0.6554
Epoch 6/10000
7906/7906 [==============================] - 24s 3ms/step - loss: 0.6440 - accuracy: 0.6555 - val_loss: 0.6440 - val_accuracy: 0.6554
Epoch 7/10000
7906/7906 [==============================] - 29s 4ms/step - loss: 0.6439 - accuracy: 0.6555 - val_

[I 2023-03-29 16:02:03,979] Trial 9 finished with value: 0.6439775824546814 and parameters: {'hidden_layer_sizes': 128, 'activation': 'sigmoid', 'hidden_layers': 2, 'output_activation': 'softmax', 'alpha': 0.0001, 'optimizer': 'SGD', 'batch_size': 32, 'max_iter': 10000}. Best is trial 4 with value: 0.5899200439453125.


In [7]:
print('Best hyperparameters: ', study.best_params)
print('Best objective value: ', study.best_value)

Best hyperparameters:  {'hidden_layer_sizes': 128, 'activation': 'tanh', 'hidden_layers': 1, 'output_activation': 'sigmoid', 'alpha': 0.0001, 'optimizer': 'Adam', 'batch_size': 64, 'max_iter': 10000}
Best objective value:  0.5899200439453125


#### **Saving study**

In [8]:
import joblib

In [9]:
save_path = './optuna_studies/mlp_study.pkl'

joblib.dump(study, save_path)

['./optuna_studies/mlp_study.pkl']